<a href="https://colab.research.google.com/github/hanucherukuri/Sentiment_analysis_using_deep_learning_word_embedding_on_yelp_reviews/blob/main/yelp_DL_SA_with_word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# loading dataset
!pip install datasets
from datasets import load_dataset

     |████████████████████████████████| 264 kB 4.9 MB/s 
     |████████████████████████████████| 118 kB 10.0 MB/s 
     |████████████████████████████████| 243 kB 10.4 MB/s 
     |████████████████████████████████| 43 kB 1.5 MB/s 
     |████████████████████████████████| 76 kB 2.9 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [ ]:
# Importing required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
! pip install nltk
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import EarlyStopping
from keras.losses import binary_crossentropy
from keras.optimizers import Adam
from keras.callbacks import History 
history = History()


In [ ]:
train_dataset = load_dataset('yelp_polarity', split = "train")
test_dataset = load_dataset('yelp_polarity', split = "test")

Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/166M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset yelp_polarity downloaded and prepared to /root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/a770787b2526bdcbfc29ac2d9beb8e820fbc15a03afd3ebc4fb9d8529de57544. Subsequent calls will reuse this data.


Reusing dataset yelp_polarity (/root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/a770787b2526bdcbfc29ac2d9beb8e820fbc15a03afd3ebc4fb9d8529de57544)


In [ ]:
wordnet = WordNetLemmatizer()

In [ ]:
def text_prep(text):
  text = text.lower()
  text = text.encode('ascii', 'ignore').decode()
  text = re.sub(r'https*\S+', ' ', text)
  text = re.sub(r'@\S+', ' ', text)
  text = re.sub(r'#\S+', ' ', text)
  text = re.sub(r'\'\w+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
  text = re.sub(r'\w*\d+\w*', '', text)
  text = re.sub(r'\s{2,}', ' ', text)
  return text

In [ ]:
# converting into a dataframe
train_df = pd.DataFrame(train_dataset)
test_df = pd.DataFrame(test_dataset)

In [ ]:
train_df['text'] = train_df.text.apply(text_prep)
test_df['text'] = test_df.text.apply(text_prep)

In [ ]:
max_features = 5000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(train_df["text"].values)
X = tokenizer.texts_to_sequences(train_df["text"].values)
X = sequence.pad_sequences(X)
Y = train_df["label"].values
print(X[:2], Y[:2])

array([[  0,   0,   0, ..., 158, 906, 227],
       [  0,   0,   0, ...,   2,  71, 603]], dtype=int32)

In [ ]:
embed_dim = 128
model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length = X.shape[1]))
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1021, 128)         640000    
_________________________________________________________________
lstm (LSTM)                  (None, 32)                20608     
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 660,641
Trainable params: 660,641
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size = 256
epochs = 2
model.fit(X, Y, validation_split=0.10, batch_size=batch_size, epochs = epochs, verbose = 1, callbacks=[history] )